**Author:** Elias Rafael Ruiz-Morales

**Institution:** Wellcome Sanger institute

**September, 2023**

---

# scVI integration of the All-infections cleaned data


In [1]:
from __future__ import print_function
import torch

import sys, os
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True'
sys.path.insert(1, '../results/scVI/')

In [2]:
# Seed for reproducibility
import numpy as np
import pandas as pd
import scanpy as sc
from typing import Tuple

# scVI imports
import scvi
from scvi.dataset import AnnDatasetFromAnnData
from scvi.inference import UnsupervisedTrainer
from scvi.models.vae import VAE

torch.manual_seed(0)
np.random.seed(0)
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)



def MovePlots(plotpattern, subplotdir):
    os.system('mkdir -p '+str(sc.settings.figdir)+'/'+subplotdir)
    os.system('mv '+str(sc.settings.figdir)+'/*'+plotpattern+'** '+str(sc.settings.figdir)+'/'+subplotdir)

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.figdir = '../results/images/'
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

sys.executable

-----
anndata     0.7.6
scanpy      1.7.0
sinfo       0.3.1
-----
PIL                 8.3.0
absl                NA
anndata             0.7.6
anyio               NA
attr                21.2.0
babel               2.9.1
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
cairo               1.20.1
certifi             2021.05.30
cffi                1.14.4
chardet             4.0.0
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
debugpy             1.3.0
decorator           5.0.9
defusedxml          0.7.1
dunamai             1.5.5
fsspec              2021.06.1
get_version         3.2
google              NA
h5py                3.3.0
idna                2.10
igraph              0.8.3
ipykernel           6.0.0
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              3.0.1
joblib              1.0.1
json5               NA
jsonschema          3.2.0
jupyter_ser

'/opt/conda/envs/scvi-singularity/bin/python'

### Loading non-normalized data

In [3]:
adata = sc.read('../results/scVI/snRNA3_raw_scVIintegrated_Trophoblast.h5ad')
adata.X.shape

(28040, 36601)

In [4]:
#raw (denoised) counts
adata.X=adata.layers["raw_counts"].copy()

### Compute the scVI latent space


In [5]:
np.unique(adata.obs['donor_id'])

array(['donor_doublets', 'snDonor_Pf0', 'snDonor_Pf1', 'snDonor_Pf2',
       'snDonor_Pf3', 'snDonor_Pf5', 'snDonor_Pf6'], dtype=object)

In [6]:
adata.obs['sample']

Pla_HDBR11923126_AAACCCAGTGGACAGT    Pla_HDBR11923126
Pla_HDBR11923126_AAACCCATCGTCTACC    Pla_HDBR11923126
Pla_HDBR11923126_AAACGAAGTGGACTGA    Pla_HDBR11923126
Pla_HDBR11923126_AAACGAATCGCAACAT    Pla_HDBR11923126
Pla_HDBR11923126_AAACGCTAGAGTCAAT    Pla_HDBR11923126
                                           ...       
Pla_HDBR13661578_TTTCATGAGTCTAGAA    Pla_HDBR13661578
Pla_HDBR13661578_TTTCATGCATTAGGCT    Pla_HDBR13661578
Pla_HDBR13661578_TTTCATGGTATGCTTG    Pla_HDBR13661578
Pla_HDBR13661578_TTTGGAGAGAGCATAT    Pla_HDBR13661578
Pla_HDBR13661578_TTTGTTGGTTAAGTCC    Pla_HDBR13661578
Name: sample, Length: 28040, dtype: category
Categories (8, object): ['Pla_HDBR11923126', 'Pla_HDBR11923127', 'Pla_HDBR13026230', 'Pla_HDBR13026231', 'Pla_HDBR13026232', 'Pla_HDBR13026233', 'Pla_HDBR13661577', 'Pla_HDBR13661578']

In [7]:
scvi.data.setup_anndata(
    adata,
    layer="raw_counts",
    batch_key='donor_id', #samples as a batch
    #donor as a covariate of the cells
    categorical_covariate_keys=["sample"] 
    #categorical_covariate_keys=['donor_souporcell',], #
    #continuous_covariate_keys=[""]
)

INFO     Using batches from adata.obs["donor_id"]                                            


Using batches from adata.obs["donor_id"]


INFO     No label_key inputted, assuming all cells have same label                           


No label_key inputted, assuming all cells have same label


INFO     Using data from adata.layers["raw_counts"]                                          


...

Computing library size prior per batch


INFO     Successfully registered anndata object containing 28040 cells, 36601 vars, 7        
         batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates   
         and 0 extra continuous covariates.                                                  


Successfully registered anndata object containing 28040 cells, 36601 vars, 7 batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates and 0 extra continuous covariates.


INFO     Please do not further modify adata until model is trained.                          


Please do not further modify adata until model is trained.


In [8]:
#---- check #layers

In [11]:
models = {}

# let's try a few values
n_latent_values = [20, 30]

for n_latent_value in n_latent_values:
    print('n_latent_value', n_latent_value)
    models[n_latent_value] = scvi.model.SCVI(adata, n_latent = n_latent_value)

n_latent_value 20
n_latent_value 30


In [12]:
models[30]

SCVI Model with the following params: 
n_hidden: 128, n_latent: 30, n_layers: 1, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: zinb, latent_distribution: normal
Training status: Not Trained

To print summary of associated AnnData, use: scvi.data.view_anndata_setup(model.adata)

In [13]:
latent_representations = {}

for n_latent_value in n_latent_values:
    print('training model for n_latent_value:', n_latent_value)
    models[n_latent_value].train()
    
    latent_representations[n_latent_value] = models[n_latent_value].get_latent_representation()
    
    adata.obsm["X_scVI_n_latent_" + str(n_latent_value)] = latent_representations[n_latent_value]
    
    curr_df = pd.DataFrame(adata.obsm["X_scVI_n_latent_" + str(n_latent_value)])
    curr_df.to_csv('../results/scVI/20230925_obsm_with_scVI_latent_representation_n_Troph_' + str(n_latent_value) + '_corrected_by_DonorSampleRaw.csv')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 20
Epoch 285/285: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 285/285 [25:42<00:00,  5.41s/it, loss=1.07e+04, v_num=1]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 30
Epoch 285/285: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 285/285 [25:08<00:00,  5.29s/it, loss=1.07e+04, v_num=1]


In [14]:
adata.write('../results/scVI/snRNA4_raw_scVIintegrated_Trophoblastonly.h5ad')

In [15]:
adata.var["feature_types"].value_counts()


Gene Expression    36601
Name: feature_types, dtype: int64

In [ ]:
adata